In [ ]:
!git clone https://bitbucket.com/critter1/omega_sygma.git /home/nugrid/omega_sygma

In [ ]:
import sys,os
sys.path.insert(1,'/home/nugrid/omega_sygma/')
os.environ["SYGMADIR"] = "/home/nugrid/omega_sygma/"
import matplotlib
%matplotlib inline

from modules.SYGMA import start_SYGMA

In [ ]:
import sys,os
#sys.path.insert(1,'/rpod3/lsiemens/omega_sygma/')
#os.environ["SYGMADIR"] = "/rpod3/lsiemens/omega_sygma/"
#Christian: change to test latest version because above commands do not work
sys.path.insert(1,'/nfs/rpod3/critter/Notebooks/SYGMA/OMEGA_SYGMA')
os.environ["SYGMADIR"] = "/nfs/rpod3/critter/Notebooks/SYGMA/OMEGA_SYGMA"
import matplotlib
%matplotlib inline

from modules.SYGMA import start_SYGMA

In [ ]:
start_SYGMA()